In [15]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
import warnings
import time
from concurrent.futures import ThreadPoolExecutor
import numpy as np

warnings.filterwarnings("ignore", message="This area is .* Overpass max query area size")

# Load data
df = pd.read_csv("updated_file_updated_idan.csv")

# 🔧 FIXED: Remove invalid coordinates (0,0) and other problematic points
print("🧹 Cleaning invalid coordinates...")
initial_count = len(df)

# Remove (0,0) coordinates
df = df[(df['X'] != 0) | (df['Y'] != 0)].copy()

# Remove coordinates that are clearly invalid for Israel
# Israel bounds approximately: lat 29.5-33.4, lon 34.2-35.9
df_converted = df.copy()
df_converted['temp_geom'] = df_converted.apply(lambda row: Point(row['X'], row['Y']), axis=1)
temp_gdf = gpd.GeoDataFrame(df_converted, geometry='temp_geom', crs="EPSG:2039")

try:
    temp_gdf_wgs84 = temp_gdf.to_crs(epsg=4326)
    temp_gdf_wgs84["temp_lat"] = temp_gdf_wgs84.geometry.y
    temp_gdf_wgs84["temp_lon"] = temp_gdf_wgs84.geometry.x
    
    # Filter valid Israel coordinates
    valid_mask = (
        (temp_gdf_wgs84['temp_lat'] >= 29.5) & 
        (temp_gdf_wgs84['temp_lat'] <= 33.4) &
        (temp_gdf_wgs84['temp_lon'] >= 34.2) & 
        (temp_gdf_wgs84['temp_lon'] <= 35.9)
    )
    
    df = df[valid_mask].copy()
    print(f"✅ Removed {initial_count - len(df)} invalid coordinates")
    print(f"📊 Remaining valid coordinates: {len(df)}")
    
except Exception as e:
    print(f"⚠️  Coordinate validation failed: {e}")
    print("Proceeding with original filtering...")

# Get unique coordinates
unique_coords = df[['X', 'Y']].drop_duplicates().copy()
print(f"🎯 Processing {len(unique_coords)} unique coordinate pairs")

unique_coords['geometry'] = unique_coords.apply(lambda row: Point(row['X'], row['Y']), axis=1)

# Convert to WGS84
gdf_unique = gpd.GeoDataFrame(unique_coords, geometry='geometry', crs="EPSG:2039").to_crs(epsg=4326)
gdf_unique["lat"] = gdf_unique.geometry.y
gdf_unique["lon"] = gdf_unique.geometry.x

# 🔧 FIXED: Better buffer calculation
buffer_deg = 0.02  # Increased buffer for better coverage

# Define features with improved tags
features = {
    "kindergarten": {"tags": {"amenity": "kindergarten"}, "threshold": 1000},
    "school": {"tags": {"amenity": "school"}, "threshold": 1000},
    "university": {"tags": {"amenity": ["university", "college"]}, "threshold": 5000},
    "bus_stop": {"tags": {"highway": "bus_stop", "public_transport": "stop_position"}, "threshold": 500},
    "train_station": {"tags": {"railway": ["station", "halt"], "public_transport": "station"}, "threshold": 5000},
    "park": {"tags": {"leisure": ["park", "garden", "recreation_ground"]}, "threshold": 800},
    "mall": {"tags": {"shop": "mall", "building": "retail"}, "threshold": 1000},
    "supermarket": {"tags": {"shop": ["supermarket", "convenience"]}, "threshold": 1000},
    "beach": {"tags": {"natural": "beach", "leisure": "beach_resort"}, "threshold": 5000},
    "place_of_worship": {"tags": {"amenity": "place_of_worship"}, "threshold": 1500}
}

# Initialize columns
for feature in features:
    gdf_unique[f"dist_to_{feature}"] = None
    gdf_unique[f"is_{feature}_within_radius"] = 0

print("🔄 Pre-fetching POIs for the entire area...")

# 🔧 FIXED: Better bbox calculation and multiple attempts
min_lat, max_lat = gdf_unique.lat.min() - buffer_deg, gdf_unique.lat.max() + buffer_deg
min_lon, max_lon = gdf_unique.lon.min() - buffer_deg, gdf_unique.lon.max() + buffer_deg

print(f"📍 Processing area: lat {min_lat:.4f} to {max_lat:.4f}, lon {min_lon:.4f} to {max_lon:.4f}")

all_pois = {}

def fetch_pois_with_retry(feature_name, config, max_retries=3):
    """Fetch POIs with multiple retry strategies"""
    
    for attempt in range(max_retries):
        try:
            # Strategy 1: Try with bbox (list format)
            if attempt == 0:
                bbox = [max_lat, min_lat, max_lon, min_lon]  # Fixed: list instead of tuple
                pois = ox.features_from_bbox(bbox=bbox, tags=config["tags"])
            
            # Strategy 2: Try with smaller chunks
            elif attempt == 1:
                print(f"  Retrying {feature_name} with chunked approach...")
                lat_chunks = np.linspace(min_lat, max_lat, 3)
                lon_chunks = np.linspace(min_lon, max_lon, 3)
                all_chunk_pois = []
                
                for i in range(len(lat_chunks)-1):
                    for j in range(len(lon_chunks)-1):
                        chunk_bbox = [lat_chunks[i+1], lat_chunks[i], lon_chunks[j+1], lon_chunks[j]]
                        try:
                            chunk_pois = ox.features_from_bbox(bbox=chunk_bbox, tags=config["tags"])
                            if not chunk_pois.empty:
                                all_chunk_pois.append(chunk_pois)
                        except:
                            continue
                
                if all_chunk_pois:
                    pois = pd.concat(all_chunk_pois, ignore_index=True)
                    pois = gpd.GeoDataFrame(pois)
                else:
                    pois = gpd.GeoDataFrame()
            
            # Strategy 3: Point-based fetch around center
            else:
                print(f"  Retrying {feature_name} with point-based approach...")
                center_lat = (min_lat + max_lat) / 2
                center_lon = (min_lon + max_lon) / 2
                radius = geodesic((min_lat, min_lon), (max_lat, max_lon)).meters / 2
                
                pois = ox.features_from_point((center_lat, center_lon), dist=radius, tags=config["tags"])
            
            # Process successful result
            if not pois.empty:
                # Filter only point geometries and polygons converted to points
                if 'geometry' in pois.columns:
                    # Convert to projected CRS first for accurate centroid calculation
                    pois_projected = pois.to_crs(epsg=2039)  # Israel TM Grid
                    
                    # Convert non-point geometries to their centroids in projected CRS
                    non_point_mask = pois_projected.geometry.type != "Point"
                    if non_point_mask.any():
                        pois_projected.loc[non_point_mask, 'geometry'] = pois_projected.loc[non_point_mask, 'geometry'].centroid
                    
                    # Convert back to WGS84
                    pois_clean = pois_projected[pois_projected.geometry.type == "Point"].to_crs(epsg=4326)
                all_pois[feature_name] = pois_clean
                print(f"  ✅ Found {len(pois_clean)} {feature_name} locations (attempt {attempt + 1})")
                return
            else:
                print(f"  ⚠️  No {feature_name} found (attempt {attempt + 1})")
                
        except Exception as e:
            print(f"  ❌ Attempt {attempt + 1} failed for {feature_name}: {str(e)[:100]}...")
            if attempt == max_retries - 1:
                print(f"  💀 All attempts failed for {feature_name}")
                all_pois[feature_name] = gpd.GeoDataFrame()

# Fetch all POIs
for feature_name, config in features.items():
    print(f"\n🔍 Fetching {feature_name}...")
    fetch_pois_with_retry(feature_name, config)
    time.sleep(1)  # Rate limiting

print("\n✅ POI pre-fetching complete!")

def process_single_point(idx):
    """Process a single coordinate point"""
    try:
        row = gdf_unique.iloc[idx]
        center = (row.lat, row.lon)
        saved_info = {}

        for feature_name, config in features.items():
            threshold = config["threshold"]
            pois = all_pois.get(feature_name, gpd.GeoDataFrame())

            if pois.empty:
                gdf_unique.at[gdf_unique.index[idx], f"dist_to_{feature_name}"] = 10000  # Default large distance
                gdf_unique.at[gdf_unique.index[idx], f"is_{feature_name}_within_radius"] = 0
                saved_info[feature_name] = "No data"
                continue

            # Spatial filtering for performance
            lat_buffer = 0.05  # ~5km buffer
            lon_buffer = 0.05
            
            mask = (
                (pois.geometry.y >= row.lat - lat_buffer) & 
                (pois.geometry.y <= row.lat + lat_buffer) &
                (pois.geometry.x >= row.lon - lon_buffer) & 
                (pois.geometry.x <= row.lon + lon_buffer)
            )

            nearby_pois = pois[mask]
            
            # If no nearby POIs in buffer, use all POIs
            search_pois = nearby_pois if not nearby_pois.empty else pois

            # Calculate distances
            distances = search_pois.geometry.apply(
                lambda geom: geodesic(center, (geom.y, geom.x)).meters
            )

            if len(distances) > 0:
                min_dist = distances.min()
                gdf_unique.at[gdf_unique.index[idx], f"dist_to_{feature_name}"] = min_dist
                gdf_unique.at[gdf_unique.index[idx], f"is_{feature_name}_within_radius"] = 1 if min_dist <= threshold else 0
                saved_info[feature_name] = f"{min_dist:.0f}m"
            else:
                gdf_unique.at[gdf_unique.index[idx], f"dist_to_{feature_name}"] = 10000
                gdf_unique.at[gdf_unique.index[idx], f"is_{feature_name}_within_radius"] = 0
                saved_info[feature_name] = "No calc"

        return idx, row['X'], row['Y'], saved_info
        
    except Exception as e:
        print(f"❌ Error processing point {idx}: {e}")
        return idx, None, None, {"error": str(e)}

# Process all points in batches
batch_size = 10  # Reduced for stability
total = len(gdf_unique)
start_time = time.time()

print(f"\n🚀 Starting processing of {total} points in batches of {batch_size}")

for batch_start in range(0, total, batch_size):
    batch_end = min(batch_start + batch_size, total)
    print(f"\n🔄 Processing batch {batch_start + 1} to {batch_end}...")

    batch_indices = list(range(batch_start, batch_end))

    # Process batch with threading
    with ThreadPoolExecutor(max_workers=3) as executor:  # Reduced workers
        futures = [executor.submit(process_single_point, idx) for idx in batch_indices]
        
        for i, future in enumerate(futures):
            try:
                idx, x, y, saved_info = future.result(timeout=30)  # 30 second timeout
                if x is not None and y is not None:
                    print(f"  ✅ Point {idx+1}/{total} | X={x:.0f}, Y={y:.0f}")
                else:
                    print(f"  ❌ Failed point {idx+1}/{total}")
            except Exception as e:
                print(f"  ⚠️  Timeout/error for point {idx+1}: {e}")

    # Progress reporting
    elapsed = time.time() - start_time
    progress = (batch_end / total) * 100
    avg_time_per_point = elapsed / batch_end if batch_end > 0 else 0
    estimated_total = avg_time_per_point * total
    remaining_time = estimated_total - elapsed

    print(f"📊 Progress: {progress:.1f}% | "
          f"Elapsed: {elapsed/60:.1f}min | ETA: {remaining_time/60:.1f}min")

    # Save intermediate results every 50 points
    if batch_end % 50 == 0 or batch_end == total:
        try:
            intermediate_df = df.merge(
                gdf_unique.drop(columns=['geometry', 'lat', 'lon']),
                on=['X', 'Y'],
                how='left'
            )
            intermediate_df.to_csv(f"intermediate_results_{batch_end}.csv", index=False)
            print(f"💾 Saved intermediate results: {batch_end} points processed")
        except Exception as e:
            print(f"⚠️  Failed to save intermediate results: {e}")

    time.sleep(1)  # Rate limiting between batches

# Final merge and save
print("\n🔄 Creating final enriched dataset...")

try:
    df_enriched = df.merge(
        gdf_unique.drop(columns=['geometry', 'lat', 'lon']),
        on=['X', 'Y'],
        how='left'
    )

    # Fill missing values with reasonable defaults
    for feature in features:
        dist_col = f"dist_to_{feature}"
        within_col = f"is_{feature}_within_radius"
        
        if dist_col in df_enriched.columns:
            df_enriched[dist_col] = df_enriched[dist_col].fillna(10000)  # 10km default
        if within_col in df_enriched.columns:
            df_enriched[within_col] = df_enriched[within_col].fillna(0)

    # Save final result
    output_filename = "enriched_environmental_data_enhanced_version_3_idan.csv"
    df_enriched.to_csv(output_filename, index=False)
    
    print(f"✅ SUCCESS! File saved as: {output_filename}")
    print(f"📊 Total records: {len(df_enriched)}")
    print(f"⏱️  Total processing time: {(time.time() - start_time)/60:.1f} minutes")

    # Print summary statistics
    print(f"\n📈 SUMMARY STATISTICS:")
    print("=" * 60)
    
    for feature in features:
        dist_col = f"dist_to_{feature}"
        within_col = f"is_{feature}_within_radius"

        if dist_col in df_enriched.columns:
            valid_dists = df_enriched[dist_col].replace([0, 10000], np.nan).dropna()
            within_count = df_enriched[within_col].sum()
            total_count = len(df_enriched)

            if len(valid_dists) > 0:
                print(f"{feature:20s} | Avg: {valid_dists.mean():6.0f}m | "
                      f"Min: {valid_dists.min():6.0f}m | "
                      f"Within: {within_count:4d}/{total_count} ({within_count/total_count*100:4.1f}%)")
            else:
                print(f"{feature:20s} | No valid distances found")

    print("=" * 60)
    print("🎉 Processing completed successfully!")

except Exception as e:
    print(f"❌ Error in final processing: {e}")
    print("Check intermediate files for partial results.")

# 🎯 BONUS: Add function for processing new single coordinates
def get_environmental_features_single_point(lat, lon, features_config=None):
    """
    Get environmental features for a single new coordinate point
    Use this function for predicting new locations!
    """
    if features_config is None:
        features_config = features
    
    result = {}
    
    for feature_name, config in features_config.items():
        try:
            # Fetch POIs around the point
            radius = min(config["threshold"] * 3, 5000)  # Search radius
            pois = ox.features_from_point((lat, lon), dist=radius, tags=config["tags"])
            
            if not pois.empty:
                # Convert to projected CRS for accurate centroid calculation
                pois_projected = pois.to_crs(epsg=2039)  # Israel TM Grid
                
                # Convert non-point geometries to their centroids in projected CRS
                non_point_mask = pois_projected.geometry.type != "Point"
                if non_point_mask.any():
                    pois_projected.loc[non_point_mask, 'geometry'] = pois_projected.loc[non_point_mask, 'geometry'].centroid
                
                # Convert back to WGS84 and filter points
                pois_points = pois_projected[pois_projected.geometry.type == "Point"].to_crs(epsg=4326)
                
                # Calculate distances
                distances = pois_points.geometry.apply(
                    lambda geom: geodesic((lat, lon), (geom.y, geom.x)).meters
                )
                
                min_dist = distances.min()
                result[f"dist_to_{feature_name}"] = min_dist
                result[f"is_{feature_name}_within_radius"] = 1 if min_dist <= config["threshold"] else 0
            else:
                result[f"dist_to_{feature_name}"] = 10000
                result[f"is_{feature_name}_within_radius"] = 0
                
        except Exception as e:
            print(f"Error fetching {feature_name}: {e}")
            result[f"dist_to_{feature_name}"] = 10000
            result[f"is_{feature_name}_within_radius"] = 0
    
    return result

print(f"\n💡 TIP: Use get_environmental_features_single_point(lat, lon) for new predictions!")

🧹 Cleaning invalid coordinates...
✅ Removed 263 invalid coordinates
📊 Remaining valid coordinates: 10739
🎯 Processing 398 unique coordinate pairs
🔄 Pre-fetching POIs for the entire area...
📍 Processing area: lat 29.5262 to 33.1037, lon 34.5370 to 35.6428

🔍 Fetching kindergarten...
  ✅ Found 1994 kindergarten locations (attempt 1)

🔍 Fetching school...
  ✅ Found 5659 school locations (attempt 1)

🔍 Fetching university...
  ✅ Found 443 university locations (attempt 1)

🔍 Fetching bus_stop...
  ✅ Found 31035 bus_stop locations (attempt 1)

🔍 Fetching train_station...
  ✅ Found 265 train_station locations (attempt 1)

🔍 Fetching park...
  ✅ Found 19826 park locations (attempt 1)

🔍 Fetching mall...
  ✅ Found 1493 mall locations (attempt 1)

🔍 Fetching supermarket...
  ✅ Found 6804 supermarket locations (attempt 1)

🔍 Fetching beach...
  ✅ Found 391 beach locations (attempt 1)

🔍 Fetching place_of_worship...
  ✅ Found 6227 place_of_worship locations (attempt 1)

✅ POI pre-fetching complete

C:\Users\idanb\AppData\Local\Temp\ipykernel_28008\972245534.py:291: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_enriched[dist_col] = df_enriched[dist_col].fillna(10000)  # 10km default


✅ SUCCESS! File saved as: enriched_environmental_data_enhanced_version_3_idan.csv
📊 Total records: 10739
⏱️  Total processing time: 3.1 minutes

📈 SUMMARY STATISTICS:
kindergarten         | Avg:   3380m | Min:     10m | Within: 4973/10739 (46.3%)
school               | Avg:   1206m | Min:     39m | Within: 7216/10739 (67.2%)
university           | Avg:   8649m | Min:     52m | Within: 4833/10739 (45.0%)
bus_stop             | Avg:    686m | Min:     11m | Within: 6475/10739 (60.3%)
train_station        | Avg:   6257m | Min:    212m | Within: 6472/10739 (60.3%)
park                 | Avg:   1000m | Min:      6m | Within: 7418/10739 (69.1%)
mall                 | Avg:   4534m | Min:     46m | Within:  901/10739 ( 8.4%)
supermarket          | Avg:   2172m | Min:     73m | Within: 2650/10739 (24.7%)
beach                | Avg:  25530m | Min:    159m | Within: 1033/10739 ( 9.6%)
place_of_worship     | Avg:   1507m | Min:      8m | Within: 7095/10739 (66.1%)
🎉 Processing completed successful